## Test the MIRI LRS distortion solution (standalone and JWST pipeline versions)  ##

### Set basic environment stuff ###

In [1]:
import os as os
import numpy as np
from numpy.testing import assert_allclose
import pdb as pdb
import pysiaf

siaf = pysiaf.Siaf('MIRI')

Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the local data directory environmental variable is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

### First we'll test the standalone distortion solution (lrs_tools) ###

In [3]:
import miricoord.miricoord.lrs.lrs_tools as lt

In [4]:
# Set to the CDP-7beta3 version (i.e., use the cdp7b3 spatial distortion)
lt.set_toolversion('cdp7beta3')

In [5]:
# Slitless prism reference point is 37.5, 300.0 in the 0-indexed subarray frame
x,y=37.5,300.
v2,v3,lam=lt.xytov2v3lam(x,y,'slitless')
v2,v3,lam

(-378.26780749377394, -344.63809783350587, array(8.4104062))

In [6]:
# Compare the location to SIAF
siaf['MIRIM_SLITLESSPRISM'].V2Ref,siaf['MIRIM_SLITLESSPRISM'].V3Ref

(-378.267807, -344.638097)

In [7]:
#Try a different point
x,y=37.5,100.
v2,v3,lam=lt.xytov2v3lam(x,y,'slitless')
v2,v3,lam

(-378.35877717232944, -344.63089262662623, array(12.89139674))

In [8]:
# Slit reference point is 325.13, 299.7 in the 0-indexed full array
x,y=325.13, 299.7
v2,v3,lam=lt.xytov2v3lam(x,y,'slit')
v2,v3,lam

(-414.3304108259057, -400.69001734056974, array(8.4104062))

In [9]:
# Compare the location to SIAF
siaf['MIRIM_SLIT'].V2Ref,siaf['MIRIM_SLIT'].V3Ref

(-414.33, -400.69)

In [10]:
#Try a different point
x,y=315,350.
v2,v3,lam=lt.xytov2v3lam(x,y,'slit')
v2,v3,lam

(-413.1854503092411, -400.78155681856504, array(6.64358458))

In [11]:
# Run a full set of tests on both slit and slitless locations
lt.testtransform()

In [12]:
# We can also use the CDP-7 version of the spatial distortion, which gives slightly different results
lt.set_toolversion('cdp7')

In [13]:
x,y=325.13, 299.7
v2,v3,lam=lt.xytov2v3lam(x,y,'slit')
v2,v3,lam

(-415.06904663622146, -400.5759204338505, array(8.4104062))

In [14]:
### Now we'll test the pipeline distortion solution (lrs_pipetools) ###

In [15]:
# Note that this only exists for CDP-7, not CDP7-beta3
import miricoord.miricoord.lrs.lrs_pipetools as lpt
lpt.version()

/Users/dlaw/anaconda3/envs/conda-drlwcs/lib/python3.6/site-packages/asdf/asdftypes.py:14: AsdfDeprecationWarning: The module asdf.asdftypes has been deprecated and will be removed in 3.0. Use asdf.types instead.
  "Use asdf.types instead.", AsdfDeprecationWarning)


'cdp7'

In [16]:
# Slit reference point is 325.13, 299.7 in the 0-indexed full array
x,y=325.13, 299.7
v2,v3,lam=lpt.xytov2v3lam(x,y,'slit')
v2,v3,lam

(-415.06904663622146, -400.57592043385057, 8.410391308079335)

In [17]:
# Test an inverse transform
xnew,ynew=lpt.v2v3lamtoxy(v2,v3,lam,'slit')
xnew,ynew

(325.13113811024033, 299.7014939850915)

In [18]:
# Test all forward/backward transforms for both slit and slitless
lpt.testtransform()